# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

## Implementation

In [163]:
import itertools
from IPython.display import display, Math
from sympy import *
from sympy.utilities.iterables import partitions


# Reserve some symbol heads
II, Li, du, dv = IndexedBase('II'), IndexedBase('Li'), IndexedBase('du'), IndexedBase('dv')

#### $\mathbb I^{\text{Symb}}$

$$
I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{n_m-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})
$$

for example, in depth $2$, $I(a_0; 0^{1-1}, a_1, 0^{3-1}, a_2, 0^{2-1}; a_3)\leftrightsquigarrow(0,1,1,3,2,2,2)$

Recall the $\Phi$ map

- $\Phi(I(a; b)) = 1,\quad \Phi(I(0; \cdots; 0)) = 0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};0))=(-1)^{n_0+\cdots+n_m-1}\Phi(I(0;0^{n_m-1},a_{i_m},\cdots,0^{n_0-1};a_{i_0})),\quad a_{i_0}\neq0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))\\
\displaystyle=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(I(a_{i_0};\cdots,a_{i_k},0^{p-1};0))\Phi(I(0,0^{q-1},a_{i_{k+1}},\cdots;a_{i_{m+1}})),\quad a_{i_0}, a_{i_{m+1}} \neq 0$

- $\Phi(I(0;0^{n_0-1},a_{i_1},0^{n_1-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))=\\
\displaystyle\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{\log^{p_0}(a_{i_{m+1}})}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\left[\frac{a_{i_2}}{a_{i_1}},\cdots,\frac{a_{i_{m+1}}}{a_{i_m}}\right]_{n_1+p_1,\cdots,n_m+p_m}$

Note: $\log^0(a_{i_{m+1}})$ is taken to be $1$ for $p_0=0$ even if $a_{i_{m+1}}=1$

Given depth $d$, the implementatio of $\Phi$ is
- $\Phi(i_0,1,i_1) = 1,\quad \Phi(0,n_0,\cdots,n_m,0) = 0$

- $\Phi(i_0, n_0, \cdots, i_m, n_m, 0)=(-1)^{n_0+\cdots+n_m-1}\Phi(0,n_m,i_m,\cdots,n_0,i_0),\quad i_0\neq0$

- $\displaystyle\Phi(i_0,n_0,\cdots,i_m,n_m,i_{m+1})=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(i_0,\cdots,i_k,p;0)\Phi(0,q,i_{k+1},\cdots;i_{m+1}),\quad i_0,i_{m+1} \neq 0$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,d+1)=\sum_{p_1+\cdots+p_m=n_0-1}(-1)^{n_0+m-1}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,d+1-i_m)$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,i_{m+1})=\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{\log^{p_0}(a_{i_{m+1}})}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,i_{m+1}-i_m)$

In [154]:
class ISymb(Indexed):
    def __new__(cls, *args):
        return super().__new__(cls, II, *args)
    def __init__(self, *args):
        # self.args[0] is the symbol head II
        self.m = len(args) // 2 - 1
        self.i = args[::2]
        self.n = args[1::2]
    def __lt__(self, other):
        pass
    def toHSymb(self, depth):
        '''
        Turn HSymb into ISymb given depth
        '''
        if self.m == 1:
            return 1
        elif self.i[0] == 0 and self.i[-1] == 0:
            return 0
        elif self.i[0] != 0 and self.i[-1] == 0:
            return (-1)^(sum(self.n) - 1) * ISymb(*self.args[:0:-1]).toHSymb(depth)
        elif self.i[0] != 0 and self.i[-1] != 0:
            return sum()
        elif self.i[0] == 0 and self.i[-1] != 0:
            pass
    def __repr__(self):
        return f"I({','.join('a_{'+str(i)+'},0^{'+str(n)+'-1}' for i, n in zip(self.i[:-1], self.n))},a_{{{self.i[-1]}}})"

In [164]:
list(partitions(5))

[{5: 1},
 {4: 1, 1: 1},
 {3: 1, 2: 1},
 {3: 1, 1: 2},
 {2: 2, 1: 1},
 {2: 1, 1: 3},
 {1: 5}]

In [157]:
ISymb(1,1,1)

I(a_{1},0^{1-1},a_{1})

In [156]:
display(Math(repr( ISymb(1,1,1) )))

<IPython.core.display.Math object>

#### $\mathbb H^{\text{Symb}}$

$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow(i_1,n_1,i_2-i_1,\cdots,i_d-i_{d-1},n_d,i_{d+1}-i_d)
$$

Or $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ so that $i_r = m_1 + \cdots + m_r$

The total ordering is $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\prec(m'_1,n'_1,\cdots,m'_{d'},n'_{d'},m'_{d'+1})$
- if: $\sum n_k < \sum n'_k$
- else if: $i_{d+1} < i'_{d'+1}$
- else: $m_{d-r} > m'_{d'-r}$ or $n_{d-r} > n'_{d'-r}$ and the entries to the right coincide

The $\partial_{r}$ of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ is
- if $d == 1$ and $n_1 == 1$: $-dv_{i_1,i_2-1}$
- else if $n_r>1$ or $d=1$: $(\cdots,n_r-1,\cdots) du_{i_r, i_{r+1}-1}$
- else if $r=d$: $-(\cdots,n_{d-1},m_d+1+m_{d+1}) dv_{i_d,i_{d+1}-1}$
- else: $-(\cdots,m_{r}+1+m_{r+1},n_{r+1},\cdots) dv_{i_r, i_{r+1}-1} + (\cdots,m_r,n_{r+1},m_{r+1}+1+m_{r+2},\cdots) (du_{i_r, i_{r+1}-1} - dv_{i_r, i_{r+1}-1})$

Since
$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow I(a_0;a_{i_1},0^{n_1-1},\cdots,a_{i_d},0^{n_d-1};a_{i_{d+1}})
$$
so `toISymb` is implemented as
$$
(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\rightsquigarrow(0,1,i_1,n_1,\cdots,i_d,n_d,i_{d+1})
$$

In [121]:
class HSymb(Indexed):
    def __new__(cls, *args):
        return super().__new__(cls, Li, *args)
    def __init__(self, *args):
        # self.args[0] is the symbol head Li
        self.d = len(args) // 2
        self.m = (0, *args[::2])
        self.n = (0, *args[1::2])
        self.i = list(itertools.accumulate(self.m))
    def __eq__(self, other):
        return self.args == other.args
    def __lt__(self, other):
        if sum(self.n) != sum(other.n):
            return sum(self.n) < sum(other.n)

        if self.i[-1] != other.i[-1]:
            return self.i[-1] < other.i[-1]

        for r in range(1, len(self.args)):
            if self.args[-r] != other.args[-r]:
                return self.m[-r] > other.m[-r]

        return False

    def toISymb(self):
        '''
        Turn HSymb into ISymb
        '''
        args = [0, 1]
        for r in range(1, self.d + 1):
            args.extend([self.i[r], self.n[r]])
        args.append(self.i[self.d + 1])
        return ISymb(*args)

    def __repr__(self):
        return f'Li{self.args[1:]}'

    def partial_differential(self, r):
        '''
        Take the partial_r of a polylogarithm
        '''
        if self.d == 1 and self.n[1] == 1:
            return -dv[self.i[1], self.i[2] - 1]
        if self.n[r] > 1 or self.d == 1:
            return HSymb(*self.args[1:2*r-1], self.n[r] - 1, *self.args[2*r+1:]) * du[self.i[r], self.i[r+1] - 1]
        elif r == self.d:
            return -HSymb(*self.args[1:-3], self.m[r] + 1 + self.m[r+1]) * dv[self.i[r], self.i[r+1] - 1]
        else:
            return -HSymb(*self.args[1:2*(r-1)], self.m[r] + 1 + self.m[r+1], *self.args[2*(r+1):]) * dv[self.i[r], self.i[r+1] - 1] +\
                    HSymb(*self.args[1:2*r-1], self.n[r+1], self.m[r+1] + 1 + self.m[r+2], *self.args[2*(r+2):]) *\
                    (du[self.i[r], self.i[r+1] - 1] - dv[self.i[r], self.i[r+1] - 1])

    def differential(self):
        '''
        Take the differential of a polylogarithm
        '''
        return sum(self.partial_differential(r) for r in range(1, self.d + 1))

#### Differential

encodings rendered by successive partial derivatives. The $\partial_{r}$-terms of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ include
- if $n_r>1$ or $d=1$: $(\cdots,n_r-1,\cdots)$
- else if $r=d$: $(\cdots,n_{d-1},m_d+1+m_{d+1})$
- else: $(\cdots,m_{r}+1+m_{r+1},n_{r+1},\cdots)$ and $(\cdots,m_r,n_{r+1},m_{r+1}+1+m_{r+2},\cdots)$

In [113]:
def differential(expr):
    '''
    Take the differential of an element in HSymb
    '''
    if expr.is_Add:
        return sum(differential(arg) for arg in expr.args)
    elif expr.is_Mul:
        return sum(expr / arg * differential(arg) for arg in expr.args if not arg.is_number)
    elif expr.is_Pow:
        return expr.args[1] * expr.args[0] ** (expr.args[1] - 1) * differential(expr.args[0])
    elif expr.is_number:
        return 0

    elif isinstance(expr, HSymb):
        return expr.differential()

#### Tensor

$$
\left(\sum_{i_1}c_{i_i}^{(1)}t_{i_1}^{(1)}\right)\otimes\cdots\otimes\left(\sum_{i_N}c_{i_N}^{(N)}t_{i_N}^{(N)}\right)=\sum_{i_1,\cdots,i_N}c_{i_i}^{(1)}\cdots c_{i_N}^{(N)} t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(N)}
$$

$$
\left(\sum_{\mathbf i}c_{\mathbf i}^{(1)}t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(1)}\right)\left(\sum_{\mathbf j}c_{\mathbf j}^{(2)}t_{j_1}^{(2)}\otimes\cdots\otimes t_{j_N}^{(2)}\right)=\sum_{\mathbf i,\mathbf j}c_{\mathbf i}^{(1)}c_{\mathbf j}^{(2)}\left(t_{i_1}^{(1)}t_{j_1}^{(2)}\right)\otimes\cdots\otimes\left(t_{i_N}^{(1)}t_{j_N}^{(2)}\right)
$$

In [114]:
class tensor():
    def __init__(self, *args):
        self.is_number = False
        # Giving coefficients and terms already
        if len(args) == 2 and all(isinstance(arg, (list, tuple)) for arg in args):
            self.c, self.t = args
        # Giving the arguments of a single raw tensor
        else:
            # expand the args first
            args = [expand(arg) for arg in args]

            self.c, self.t = [], []
            for comp in list(itertools.product(*map(tensor._split_coef, args))):
                c, t = zip(*comp)
                self.c.append(prod(c))
                self.t.append(t)

    @staticmethod
    def _split_coef(expr):
        if expr.is_Add:
            return [tensor._split_coef(arg)[0] for arg in expr.args]
        elif expr.is_number:
            return [[expr, 1]]
        elif expr.is_Mul:
            coef = prod(arg for arg in expr.args if arg.is_number)
            return [[coef, expr / coef]]
        else:
            return [[1, expr]]

    def __add__(self, other):
        return tensor(self.c + other.c, self.t + other.t)
    def __neg__(self):
        return tensor(list(map(lambda x: -x, self.c)), self.t)
    def __sub__(self, other):
        return self + (-other)
    def __rmul__(self, other):
        # Scalar multiplication (scalar on the left side)
        if isinstance(other, (int, float)) or other.is_number:
            return tensor([c * other for c in self.c], self.t)
        return self * other
    def __mul__(self, other):
        if isinstance(other, (int, float)) or other.is_number:
            return tensor([c * other for c in self.c], self.t)
        return tensor(list(map(prod, itertools.product(self.c, other.c))),
                      [tensor._mul(t1, t2) for t1, t2 in itertools.product(self.t, other.t)])

    @staticmethod
    def _mul(t1, t2):
        if not isinstance(t1, (list, tuple)) or not isinstance(t2, (list, tuple)):
            raise ValueError('Must be a list or a tuple')
        if len(t1) != len(t2):
            raise ValueError('Dimension Mismatch')
        return tuple(map(prod, zip(t1, t2)))

    def __repr__(self):
        ct = []
        for c, t in zip(self.c, self.t):
            if c == 1:
                ct.append(f"{t}")
            elif c == -1:
                ct.append(f"-{t}")
            else:
                ct.append(f"{c}*{t}")

        return ' + '.join(ct)

#### Wedge

In [ ]:
class wedge:
    def __init__(self, *comp):
        self.comp = comp
    def __repr__(self):
        return f'{self.comp}'
    def __repr__(self):
        ct = []
        for c, t in zip(self.c, self.t):
            if c == 1:
                ct.append(f"{t}")
            elif c == -1:
                ct.append(f"-{t}")
            else:
                ct.append(f"{c}*{t}")

        return ' + '.join(ct)

#### Variation matrix over $S_d(\mathbb C)$

Recall the *complementary entry* of $(-1)^kI(0;a_{i_1},0^{m_{i_1}-1},\cdots,a_{i_k},0^{m_{i_k}-1};1)$ with respect to $(-1)^lI(0;a_{j_1},0^{p_{j_1}-1},\cdots,a_{j_l},0^{p_{j_l}-1};1)$ is
$$
(-1)^{l-k} I^{\sigma_{i_1}\sigma_0^{m_{i_1}-1}\cdots\sigma_{i_k}\sigma_0^{m_{i_k}-1}}(0;a_{j_1},0^{p_{j_1}-1},\cdots,a_{j_l},0^{p_{j_l}-1};1)
$$

$\sigma_{i_1}\sigma_0^{m_1-1}\cdots\sigma_{i_k}\sigma_0^{m_k-1}$ should correspond to $(0,1,i_1,m_1,\cdots, i_k,m_k)$

Suppose $\{i_1,\cdots,i_k\}=\{j_{q_1},\cdots,j_{q_k}\}$ is a subsequence of $\{j_1,\cdots,j_l\}$ (otherwise the complementary entry is 0), then the complementary entry is

$$
(-1)^{l-k}I(0;\cdots;a_{j_{q_1}})\left(\prod_{r=1}^{k-1} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})\right)I^{\sigma_0^{m_k-1}}(a_{j_{q_k}};\cdots;1)
$$

Assume
- $a_0=0$, $a_{d+1}=1$
- $i_0=j_0=0$, $i_{k+1}=j_{l+1}=d+1$ so that $q_0=0$, $q_{k+1}=l$,
- $m_0=1$

then the complementary entry is
$$
(-1)^{l-k}\prod_{r=0}^{k} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})
$$
If $m_r > 1$, it is equal to
$$
I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})=\sum_{\substack{q_r\leq s<q_{r+1}-1 \\ m_r \leq p_s}}\sum_{u+v=p_s-m_r}I(a_{j_{q_r}};\cdots,a_{j_s},0^{u};0)I(0;0^{v},a_{j_{s+1}},\cdots;a_{j_{q_{r+1}}})
$$



`complementaryEntry(w1, w2)`: the complementary entry of $w_1 = (0,1,j_1,p_1,\cdots,j_l,p_l)$ and $w_2 = (0,1,i_1,m_1,\cdots,i_k,m_k)$. Which should be the product of `Isigma(r)` from $r=0$ to $k$

In [ ]:
class VariationMatrix:
    def __init__(self, *args):
        self.args = args
    def firstColumn(self):
        pass
    def complementaryEntry(w1, w2):
        pass
    def __repr__(self):
        return f'{self.args}'